In [87]:
%pip install crewai langchain_openai langchain_community python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [88]:

from bs4 import BeautifulSoup
from typing import List



In [89]:

from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.tools import tool
import re
from langchain_community.document_loaders import PyMuPDFLoader
import requests


In [90]:
# Load your OPENAI_API_KEY from your .env file
load_dotenv()
# The model for the agents
model = ChatOpenAI(model_name="gpt-4o", temperature=0.8)
original_url = "http://ezshop.ca"

# Agents

amora_books_categories = [
    {
        "name": "Fiction Favorites",
        "url": "https://amorabooks.com/category/fiction-favorites"
    },
    {
        "name": "Non-Fiction Narratives",
        "url": "https://amorabooks.com/category/non-fiction-narratives"
    },
    {
        "name": "Mystery & Suspense",
        "url": "https://amorabooks.com/category/mystery-suspense"
    },
    {
        "name": "Sci-Fi & Fantasy Realms",
        "url": "https://amorabooks.com/category/scifi-fantasy-realms"
    }]


# agent_semantic_category_analyzer = Agent(
#     role='Semantic Category Analyzer',
#     goal='Identify semantically relevant categories',
#     backstory="""You are responsible for analyzing all categories on the live website 
#     and finding semantically relevant categories to the one we are writing about. 
#     You need to return four categories that are semantically related to the category 
#     as well as the links for each category.""",
#     tools=[get_category_urls],  # Add the updated custom scraping tool
#     llm=model,
#     max_iter=15,
#     max_execution_time=60,
#     verbose=True,
#     allow_delegation=True,
#     cache=True
# )

agent_ecommerce_content_creator = Agent(
    role='E-commerce Content Creator',
    goal='Write comprehensive text for the category',
    backstory="""You are responsible for receiving information about the website 
    and the category we are writing the description for. You then write a comprehensive 
    and engaging text for the category.""",
    llm=model,
    max_iter=15,
     max_execution_time=60,
    verbose=True,
    allow_delegation=True,
    cache=True
)

agent_link_integration_specialist = Agent(
    role='Link Integration Specialist',
    goal='Create unique interlinking sentences',
    backstory="""You are responsible for receiving the category links and text, 
    creating four unique interlinking sentences. Each sentence must have one 
    semantically related category name linked to its respective URL provided 
    by the Semantic Category Analyzer. You must avoid redundancy and overlap.""",

    llm=model,
    max_iter=15,
    max_execution_time=60,
    verbose=True,
    allow_delegation=True,
    cache=True
)

agent_content_integration_manager = Agent(
    role='Content Integration Manager',
    goal='Combine text and interlinking sentences into a cohesive description',
    backstory="""You are responsible for receiving the category text from 
    the E-commerce Content Creator and the interlinking sentences from the 
    Link Integration Specialist. You need to combine them into a cohesive 
    and engaging category description.""",
    llm=model,
    max_iter=15,
    max_execution_time=60,
    verbose=True,
    allow_delegation=True,
    cache=True
)


# Tasks


In [91]:


# Task for analyzing categories
# task_analyze_categories = Task(
#     description="""Analyze categories on the live website and identify semantically relevant ones. url = {url}""",
#     expected_output='List of four categories that are semantically related along with their links.',
#     agent=agent_semantic_category_analyzer,  # Ensure this is correctly defined in your system
#     tools=[get_category_urls],  # Use the custom scraping tool to fetch category URLs
#     input_schema={'url': str},  # Define the expected input schema, e.g., a URL of the website
#     output_schema=List[dict]    # Define the expected output schema
# )

# Task for writing category text
task_write_category_text = Task(
    description='Write a comprehensive text for the category',
    expected_output='A well-written category text',
    agent=agent_ecommerce_content_creator
    # No tools specified, assuming text generation is done by LLM
)

# Task for creating interlinking sentences
task_create_interlinking_sentences = Task(
    description='Create four unique interlinking sentences for the category text {input}',
    expected_output='Four interlinking sentences that fit the category text',
    agent=agent_link_integration_specialist,
)

# Task for integrating content
task_integrate_content = Task(
    description='Combine the category text and interlinking sentences into a cohesive description',
    expected_output='A perfect category description',
    agent=agent_content_integration_manager
    # No tools specified, assuming integration is done by LLM
)


In [94]:
crew = Crew(
    agents=[
        agent_link_integration_specialist
    ],
    tasks=[
        task_create_interlinking_sentences,
    ],
    verbose=True
)

# Start the crew's execution with the URL input for category analysis
result = crew.kickoff(inputs={"input": "amora_books_categories"})
print("Task Analyze Categories Result:")
print(result)

2024-08-06 16:22:59,040 - 8507938048 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-06 16:22:59][DEBUG]: == Working Agent: Link Integration Specialist
 [2024-08-06 16:22:59][INFO]: == Starting Task: Create four unique interlinking sentences for the category text amora_books_categories


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer: 

1. Explore our vast selection of **[fiction books](http://example.com/fiction_books)** to dive into imaginative worlds and captivating stories.
2. Enhance your knowledge with our comprehensive collection of **[non-fiction books](http://example.com/non_fiction_books)** that cover a variety of real-world topics.
3. Discover timeless tales and literary masterpieces in our **[classics books](http://example.com/classics_books)** section.
4. Delight young readers with engaging stories from our **[children's books](http://example.com/childrens_books)** collection, perfect for fostering a love of reading.

> Finished chain.
 [2024-08-06 16:23:03][DEBUG]: == [Link Integration Specialist] Task out